# Day 25: The Halting Problem

Following the twisty passageways deeper and deeper into the CPU, you finally reach the core of the computer. Here, in the expansive central chamber, you find a grand apparatus that fills the entire room, suspended nanometers above your head.

You had always imagined CPUs to be noisy, chaotic places, bustling with activity. Instead, the room is quiet, motionless, and dark.

Suddenly, you and the CPU's garbage collector startle each other. "It's not often we get many visitors here!", he says. You inquire about the stopped machinery.

"It stopped milliseconds ago; not sure why. I'm a garbage collector, not a doctor." You ask what the machine is for.

"Programs these days, don't know their origins. That's the Turing machine! It's what makes the whole computer work." You try to explain that Turing machines are merely models of computation, but he cuts you off. "No, see, that's just what they want you to think. Ultimately, inside every CPU, there's a Turing machine driving the whole thing! Too bad this one's broken. We're doomed!"

You ask how you can help. "Well, unfortunately, the only way to get the computer running again would be to create a whole new Turing machine from scratch, but there's no way you can-" He notices the look on your face, gives you a curious glance, shrugs, and goes back to sweeping the floor.

You find the Turing machine blueprints (your puzzle input) on a tablet in a nearby pile of debris. Looking back up at the broken Turing machine above, you can start to identify its parts:

- A tape which contains 0 repeated infinitely to the left and right.
- A cursor, which can move left or right along the tape and read or write values at its current position.
- A set of states, each containing rules about what to do based on the current value under the cursor.

Each slot on the tape has two possible values: 0 (the starting value for all slots) and 1. Based on whether the cursor is pointing at a 0 or a 1, the current state says what value to write at the current position of the cursor, whether to move the cursor left or right one slot, and which state to use next.

In [1]:
import io, re
from collections import defaultdict


def match(template, s):
    m = re.match(template, s)
    if m is None:
        raise Exception('Expected %r, got %r.' % (template, s))
    return m.group(1)


def parse(f):
    if isinstance(f, str):
        f = io.StringIO(f.strip())

    # read header
    start_state = match(r'^Begin in state (\w+).$', f.readline())
    steps_until_checksum = int(
        match(r'^Perform a diagnostic checksum after (\d+) steps.$',
              f.readline()))
    assert not f.readline().strip()

    # read rules
    rules = defaultdict(dict)
    while f:
        state = match(r'^In state (\w+):$', f.readline())
        while True:
            line = f.readline()
            if not line.strip():
                break
            value = int(match(r'^  If the current value is (\d+):$', line))
            new_value = int(
                match(r'^    - Write the value (\d+).$', f.readline()))
            move_to = match(r'^    - Move one slot to the (\w+).$',
                            f.readline())
            new_state = match(r'^    - Continue with state (\w+).$',
                              f.readline())
            rules[state][value] = {
                'new_value': new_value,
                'move_to': move_to,
                'new_state': new_state
            }
        if line == '':
            break

    return {
        'start_state': start_state,
        'steps_until_checksum': steps_until_checksum,
        'rules': rules
    }


def run(blueprint):
    current_state = blueprint['start_state']
    current_slot = 0
    tape = defaultdict(int)
    while True:
        yield {'tape': tape, 'slot': current_slot, 'state': current_state}

        # look up rule
        current_value = tape[current_slot]
        rule = blueprint['rules'][current_state][current_value]

        # update tape, state, slot
        tape[current_slot] = rule['new_value']
        current_state = rule['new_state']
        assert rule['move_to'] in ['left', 'right']
        current_slot += 1 if rule['move_to'] == 'right' else -1

For example, suppose you found the following blueprint:

In [2]:
example = parse("""
Begin in state A.
Perform a diagnostic checksum after 6 steps.

In state A:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state B.
  If the current value is 1:
    - Write the value 0.
    - Move one slot to the left.
    - Continue with state B.

In state B:
  If the current value is 0:
    - Write the value 1.
    - Move one slot to the left.
    - Continue with state A.
  If the current value is 1:
    - Write the value 1.
    - Move one slot to the right.
    - Continue with state A.
""")

{'rules': defaultdict(dict,
             {'A': {0: {'move_to': 'right', 'new_state': 'B', 'new_value': 1},
               1: {'move_to': 'left', 'new_state': 'B', 'new_value': 0}},
              'B': {0: {'move_to': 'left', 'new_state': 'A', 'new_value': 1},
               1: {'move_to': 'right', 'new_state': 'A', 'new_value': 1}}}),
 'start_state': 'A',
 'steps_until_checksum': 6}

Running it until the number of steps required to take the listed diagnostic checksum would result in the following tape configurations (with the cursor marked in square brackets):

In [3]:
tm = run(example)
tape = lambda s: [s['tape'][i] for i in range(-3, 3)]

# ... 0  0  0 [0] 0  0 ... (before any steps; about to run state A)
s = next(tm)
assert tape(s) == [0, 0, 0, 0, 0, 0] and s['slot'] == 0 and s['state'] == 'A'

# ... 0  0  0  1 [0] 0 ... (after 1 step;     about to run state B)
s = next(tm)
assert tape(s) == [0, 0, 0, 1, 0, 0] and s['slot'] == 1 and s['state'] == 'B'

# ... 0  0  0 [1] 1  0 ... (after 2 steps;    about to run state A)
s = next(tm)
assert tape(s) == [0, 0, 0, 1, 1, 0] and s['slot'] == 0 and s['state'] == 'A'

# ... 0  0 [0] 0  1  0 ... (after 3 steps;    about to run state B)
s = next(tm)
assert tape(s) == [0, 0, 0, 0, 1, 0] and s['slot'] == -1 and s['state'] == 'B'

# ... 0 [0] 1  0  1  0 ... (after 4 steps;    about to run state A)
s = next(tm)
assert tape(s) == [0, 0, 1, 0, 1, 0] and s['slot'] == -2 and s['state'] == 'A'

# ... 0  1 [1] 0  1  0 ... (after 5 steps;    about to run state B)
s = next(tm)
assert tape(s) == [0, 1, 1, 0, 1, 0] and s['slot'] == -1 and s['state'] == 'B'

# ... 0  1  1 [0] 1  0 ... (after 6 steps;    about to run state A)
s = next(tm)
assert tape(s) == [0, 1, 1, 0, 1, 0] and s['slot'] == 0 and s['state'] == 'A'

The CPU can confirm that the Turing machine is working by taking a diagnostic checksum after a specific number of steps (given in the blueprint). Once the specified number of steps have been executed, the Turing machine should pause; once it does, count the number of times 1 appears on the tape. In the above example, the diagnostic checksum is 3.

In [4]:
def checksum(blueprint):
    for i, s in enumerate(run(blueprint)):
        if i == blueprint['steps_until_checksum']:
            break
    return sum(s['tape'].values())


assert checksum(example) == 3

Recreate the Turing machine and save the computer! What is the diagnostic checksum it produces once it's working again?

In [5]:
%%time
puzzle = parse(open('25.input'))
print(checksum(puzzle))

5744
CPU times: user 8.12 s, sys: 61.6 ms, total: 8.19 s
Wall time: 8.24 s
